In [105]:
import openpyxl

In [106]:
filename = 'Tariff_Input_Template_DEP NC_res.xlsx'
wb = openpyxl.load_workbook(filename, data_only=True, read_only=True)

In [107]:
array_ranges = [
    'coincident_adjustments',
    'coincident_levels',
    'coincident_rates',
    'd_adjustments',
    'd_levels',
    'd_rates',
    'e_adjustments',
    'e_levels',
    'e_rates',
    'e_sell_rates', #TODO: Find the appropriate key for this field!!!
]
matrix_ranges = [
    'coincident_12by24',
    'd_wkday_12by24',
    'd_wkend_12by24',
    'e_wkday_12by24',
    'e_wkend_12by24',
]
single_cell_ranges = [
    'comments',
    'demand_rate_unit',
    'description',
    'eia_id',
    'energy_rate_unit',
    'fixed_charge',
    'fixedchargeunits',
    'kWh_useage_max',
    'kWh_useage_min',
    'mincharge',
    'minchargeunits',
    'name',
    'rescom',
    'peak_kW_capacity_max',
    'peak_kW_capacity_min',
    'source',
    'urdb_id',
    'utility',
    'voltage_category',
]
aux_data_ranges = [ # These will be used to reshape the rates and not be part of the tariff.
    'coincident_periods',
    'coincident_tiers',
    'd_periods',
    'd_tiers',
    'e_periods',
    'e_tiers',
]

tariff_dict = {} #initialize the tariff dictionary
aux_vars = {} #auxiliary variables that will be used to modify tariff variables

In [108]:
for range_name in array_ranges:
    sheetname, cell_range = list(wb.defined_names[range_name].destinations)[0]
    sheet = wb[sheetname]
    cells = sheet[cell_range]
    vals = []
    for c in cells:
        if c[0].value is None:
            break
        vals.append(c[0].value)
    tariff_dict[range_name] = vals

In [109]:
for range_name in matrix_ranges:
    sheetname, cell_range = list(wb.defined_names[range_name].destinations)[0]
    sheet = wb[sheetname]
    cells = sheet[cell_range]
    vals = []
    for i, row in enumerate(cells):
        new_row = []
        for  j, c in enumerate(row):
            if c.value is None:
                break
            new_row.append(c.value-1)
        if len(new_row)==0:
            break
        vals.append(new_row)
    tariff_dict[range_name] = vals

In [110]:
for range_name in single_cell_ranges:
    sheetname, cell_range = list(wb.defined_names[range_name].destinations)[0]
    sheet = wb[sheetname]
    cells = sheet[cell_range]
    tariff_dict[range_name] = cells.value

In [111]:
sheetname, cell_range = list(wb.defined_names['d_periods'].destinations)[0]
sheet = wb[sheetname]
period_cells = (c[0].value for c in sheet[cell_range] if c[0].value is not None)
d_periods = max(period_cells)

sheetname, cell_range = list(wb.defined_names['d_tiers'].destinations)[0]
sheet = wb[sheetname]
tier_cells = (c[0].value for c in sheet[cell_range] if c[0].value is not None)
d_tiers = max(tier_cells)

demand_reshaped = [[0 for _ in range(d_periods)] for _ in range(d_tiers)]
demand_prices = tariff_dict.pop('d_rates')
adjustments = tariff_dict.pop('d_adjustments')
for i in range(len(adjustments)):
    demand_prices+=tariff_dict['d_adjustments'][i]
for i,j in zip(tier_cells, period_cells):
    demand_reshaped[i-1][j-1] = demand_prices.pop(0)
tariff_dict['d_tou_exists'] = d_periods>1 and len(demand_prices)>0 #TODO: Ask Jake if this is correct!
tariff_dict['d_flat_exists'] = not tariff_dict['d_tou_exists'] and len(demand_prices)>0
if tariff_dict['d_tou_exists']:
    tariff_dict['d_tou_levels'] = tariff_dict.pop('d_levels')
    tariff_dict['d_tou_prices'] = demand_reshaped
    tariff_dict['d_tou_n'] = d_periods
elif tariff_dict['d_flat_exists']:
    tariff_dict['d_flat_levels'] = tariff_dict.pop('d_levels')
    tariff_dict['d_flat_prices'] = demand_reshaped
    tariff_dict['d_tou_n'] = 0
else:
    tariff_dict['d_flat_levels'] = []
    tariff_dict['d_flat_prices'] = []
    tariff_dict['d_tou_levels'] = []
    tariff_dict['d_tou_prices'] = []
    tariff_dict['d_tou_n'] = 0

In [112]:
sheetname, cell_range = list(wb.defined_names['coincident_periods'].destinations)[0]
sheet = wb[sheetname]
period_cells = (c[0].value for c in sheet[cell_range] if c[0].value is not None)
coincident_periods = max(period_cells)

sheetname, cell_range = list(wb.defined_names['coincident_tiers'].destinations)[0]
sheet = wb[sheetname]
tier_cells = (c[0].value for c in sheet[cell_range] if c[0].value is not None)
coincident_tiers = max(tier_cells)

coincident_reshaped = [[0 for _ in range(coincident_periods)] for _ in range(coincident_tiers)]
coincident_prices = tariff_dict.pop('coincident_rates')
adjustments = tariff_dict['coincident_adjustments']
for i in range(len(adjustments)):
    coincident_prices+=adjustments[i]
for i,j in zip(tier_cells, period_cells):
    coincident_reshaped[i-1][j-1] = coincident_prices.pop(0)
tariff_dict['coincident_prices'] = coincident_reshaped
    

In [ ]:
sheetname, cell_range = list(wb.defined_names['e_periods'].destinations)[0]
sheet = wb[sheetname]
period_cells = [c[0].value for c in sheet[cell_range] if c[0].value is not None]
e_periods = max(period_cells)
print(e_periods)

sheetname, cell_range = list(wb.defined_names['e_tiers'].destinations)[0]
sheet = wb[sheetname]
tier_cells = [c[0].value for c in sheet[cell_range] if c[0].value is not None]
e_tiers = max(tier_cells)
energy_reshaped = [[0 for _ in range(e_periods)] for _ in range(e_tiers)]
energy_prices = tariff_dict.pop('e_rates')
adjustments = tariff_dict.pop('e_adjustments')
for i in range(len(adjustments)):
    energy_prices+=adjustments[i]
for i,j in zip(tier_cells, period_cells):
    energy_reshaped[i-1][j-1] = energy_prices.pop(0)
tariff_dict['e_tou_exists'] = e_periods>1
tariff_dict['e_exists'] = not tariff_dict['e_tou_exists']
tariff_dict['e_prices'] = energy_reshaped
tariff_dict['e_n'] = e_periods

In [114]:
tariff_dict

{'coincident_adjustments': [],
 'coincident_levels': [],
 'd_levels': [],
 'e_levels': [],
 'e_sell_rates': [0.034, 0.034, 0.034, 0.034, 0.034, 0.034],
 'coincident_12by24': [],
 'd_wkday_12by24': [[0,
   0,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   1,
   1,
   0,
   2,
   2,
   0,
   0,
   0,
   2,
   2,
   2,
   0,
   0],
  [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0],
  [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0],
  [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0],
  [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 2, 2, 